In [1]:
import bernielib as bl

In [2]:
bl.listSerialPorts()

['COM7', 'COM18', 'COM20']

In [3]:
ber = bl.robot(cartesian_port_name='COM18', pipette_port_name='COM7', misc_port_name='COM20')

In [4]:
ber.robotHome()

In [5]:
ber.pipetteHome()

# Function for beads amount calculations

In [6]:
# Those numbers are obtained 20041601, in Arnielib
def calcBeadsVol(sample_vol, DNA_size_cutoff):
    multiplier = 4.99325349e-01 - 9.91043764e+00 / DNA_size_cutoff + 2.57585836e+04 / DNA_size_cutoff ** 2
    return sample_vol * multiplier

In [7]:
calcBeadsVol(200, 100)

595.2158665200001

# Creating samples

In [8]:
def createReagentSample(type_name, sample_name, pos_col, pos_row, v):
    sample_type = bl.sample_type(type_name)
    sample = bl.sample(sample_name, sample_type)
    sample.place(ber.reagents_rack, pos_col, pos_row)
    sample.setVolume(v)
    return sample

In [9]:
def createSampleList(v_list, col):
    sample_type = bl.sample_type('eppendorf')
    sample_number = 0
    sample_list = []
    for v in v_list:
        sample = bl.sample('sample'+str(sample_number), sample_type)
        sample.place(ber.samples_rack, col, sample_number)
        sample.setVolume(v)
        sample_list.append(sample)
        sample_number += 1
    return sample_list

In [10]:
beads = createReagentSample('5ml', 'beads', 0, 0, 1000)
EtOH = createReagentSample('50ml', 'EtOH', 1, 0, 50000)
eluent = createReagentSample('50ml', 'eluent', 2, 0, 50000)
waste = createReagentSample('50ml', 'waste', 0, 1, 0)
water = createReagentSample('50ml', 'water', 1, 1, 50000)
EtOH80pct = createReagentSample('50ml', 'EtOH80pct', 2, 1, 50000)

In [11]:
samples_v_list = [100, 50, 200]

In [12]:
samples_cleanup_list = createSampleList([0, 0, 0], 1)

In [13]:
# Only use if the tubes were filled manually or if you are restarting
samples_cleanup_list = createSampleList(samples_v_list, 1)

# Pre-filling the samples with water to mock the conditions

In [13]:
ber.pickUpNextTip()

In [14]:
ber.move(z=0)

In [15]:
for sample, v in zip(samples_cleanup_list, samples_v_list):
    ber.transferLiquid(water, sample, v)

In [15]:
samples_cleanup_list[0].getVolume()

0

# Purification settings

In [16]:
upper_cutoff_list = [1000, 700, 300]
lower_cutoff_list = [300, 200, 100]

# Performing upper cutoff

In [18]:
# Upper cutoff beads volume calculation
upper_cut_v_list = []
for vol, size in zip(samples_v_list, upper_cutoff_list):
    v_beads = calcBeadsVol(vol, size)
    upper_cut_v_list.append(v_beads)

In [19]:
upper_cut_v_list

[51.517349495999994, 26.886805945102036, 150.49940826222223]

In [20]:
ber.moveMagnetsAway()

In [21]:
for v, sample in zip(upper_cut_v_list, samples_cleanup_list):
    print ("Transferring beads to sample ", sample.name, " of volume ", str(v))
    ber.transferLiquid(beads, sample, v)
    v_sample = sample.getVolume()
    v_updown = v_sample / 2
    if v_updown > 200:
        v_updown = 200
    ber.pipetteUpAndDown(sample, v_updown, 5)

Transferring beads to sample  sample0  of volume  51.517349495999994
Transferring beads to sample  sample1  of volume  26.886805945102036
Transferring beads to sample  sample2  of volume  150.49940826222223


In [24]:
830 // 200

4

In [25]:
830.1 // 200

4.0

In [26]:
51.517349495999994 // 200

0.0

In [24]:
ber.dumpTipToPosition(1, 2)

In [27]:
i=0

In [28]:
if i:
    print ('yes')
else:
    print ('no')

no


In [27]:
beads.purge()

In [17]:
ber.uptakeLiquid(beads, 200)

In [18]:
ber.dispenseLiquid(beads, 200, blow_extra=True)

In [17]:
ber.uptakeLiquid(beads, 200)

In [18]:
beads._isLowVolumeUptakeNeeded(205)

True

In [19]:
beads.getCloseToBottomVol()

91.85

In [20]:
beads._settingPresent('close_to_bottom_volume')

True

In [21]:
beads._getSetting('close_to_bottom_volume')

91.85

In [19]:
ber.pipetteUpAndDown(beads, 200, 3)

In [16]:
ber.uptakeLiquid(beads, 200, lag_vol=0)

In [18]:
ber.dispenseLiquid(beads, 200, plunger_retract=False, move_up_after=False, in_place=True)

In [21]:
ber.uptakeLiquid(beads, 100, lag_vol=0, in_place=True)

In [25]:
beads.calcAbsLiquidLevelFromVol(100, added_length=ber._calcExtraLength())

87.935

In [26]:
beads.calcNormalPipettingZ(100, 0, added_length=ber._calcExtraLength())

78.95